In [68]:
import math
import time
import numpy as np
import sounddevice as sd
import plotly.express as px

In [101]:
SR = 44100
sd.default.samplerate = SR

def callback(outdata: np.ndarray, frames: int): pass

current_sample = 0
def master_callback(outdata: np.ndarray, frames: int, time, status):
    global current_sample
    if status: 
        print(status)
    callback(outdata, frames)
    current_sample += frames

stream = sd.OutputStream(channels=1, callback=master_callback)

In [105]:
class Pattern:
    def __init__(self, length: float):
        self.length = length
        self.length_sr = int(self.length * SR)
        self.notes = []
        self.buffer = None
        
    def play(self, time: float, note: float, length: float = 1., volume: float = 0.1):
        self.notes.append({
            "time": time,
            "start": int(time * SR),
            "end": int((time + length) * SR),
            "note": note,
            "freq": 440 + note * 100,
            "volume": volume,
        })
        
    def render(self):
        self.buffer = np.zeros(self.length_sr, dtype=np.float)
        for n in self.notes:
            samples = n["end"] - n["start"]
            end_freq = n["freq"] * samples / SR
            nbuf = np.sin(np.linspace(0, end_freq * 2. * math.pi, samples))
            nbuf *= np.linspace(1, 0, samples)
            nbuf *= n["volume"]
            self.buffer[n["start"]:min(self.length_sr, n["end"])] += nbuf[:min(samples, self.length_sr - n["start"])]
        
    def calc_callback(self, frames: int) -> np.ndarray:
        start_sample = current_sample % int(self.length * SR)
        end_sample = start_sample + frames
        buf = self.buffer[start_sample:end_sample]
        if buf.shape[0] < frames:
            buf = np.concatenate([buf, self.buffer[:frames - buf.shape[0]]])
        return buf.reshape(frames, 1)
        #for n in self.notes:
        #    if n["start"] <= start_sample and n["end"] > start_sample
    
    def plot(self):
        graph = [
            (n["time"], n["note"], n["volume"] * 5 + 1)
            for n in self.notes
        ]
        graph.sort(key=lambda n: n[0])
        px.scatter(
            x=[n[0] for n in graph], 
            y=[n[1] for n in graph], 
            size=[n[2] for n in graph]
        ).show()
        
pattern = Pattern(3)
for i in range(10):
    pattern.play(i/10*3, i%5%3, .7)
pattern.play(1, 2, 1)
pattern.render()
pattern.plot()

In [103]:
def callback(outdata: np.ndarray, frames: int):
    #f1 = current_sample / SR * 440 * math.pi * 2
    #f2 = (current_sample + frames) / SR * 440 * math.pi * 2
    #outdata[:] = np.sin(np.linspace(f1, f2, frames)).reshape(frames, 1)
    outdata[:] = pattern.calc_callback(frames)

In [104]:
stream.start()

output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow
output underflow


In [106]:
stream.stop()